# Change of Chararacter CHOCH

### Load the data

In [1]:
# 導入必要的庫
import pandas as pd  # 用於數據框操作
import pandas_ta as ta  # 用於技術分析指標
import numpy as np  # 用於數學運算
import plotly.graph_objects as go  # 用於繪製圖表
from scipy import stats  # 用於統計分析

# 從CSV文件中讀取比特幣價格數據
df = pd.read_csv("BTCUSDT_15m_2023-4-2023-10_Monthly.csv")

# 過濾掉交易量（volume）為0的數據行
df = df[df['volume'] != 0]

# 重設數據框的索引
df.reset_index(drop=True, inplace=True)

# 計算相對強度指標（RSI）
df['RSI'] = ta.rsi(df.close, length=12)

# 計算指定長度（length=150）的指數移動平均（EMA）
df['EMA'] = ta.ema(df.close, length=150)

# 顯示數據框的最後幾行
df.tail()

# 選擇前5000行數據進行進一步分析
df = df[0:5000]

In [2]:
df

,Gmt time,open,high,low,close,volume,RSI,EMA
0,2023-04-01 08:00:00,28454.8,28491.1,28438.0,28491.1,2728.658,NaN,NaN
1,2023-04-01 08:15:00,28491.0,28543.3,28456.9,28479.5,3450.581,NaN,NaN
2,2023-04-01 08:30:00,28479.5,28495.7,28401.0,28433.2,2821.510,NaN,NaN
3,2023-04-01 08:45:00,28433.1,28444.4,28401.0,28430.1,1972.972,NaN,NaN
4,2023-04-01 09:00:00,28430.1,28454.9,28400.0,28423.0,2091.700,NaN,NaN
...,...,...,...,...,...,...,...,...
4995,2023-05-23 08:45:00,26900.1,26901.0,26782.6,26815.5,4390.227,41.139183,26852.268481
4996,2023-05-23 09:00:00,26815.4,26840.2,26791.3,26829.3,2265.815,44.172873,26851.964263
4997,2023-05-23 09:15:00,26829.4,26999.0,26829.3,26967.9,5791.190,64.320867,26853.499836
4998,2023-05-23 09:30:00,26968.0,27110.0,26963.7,27019.3,17198.757,68.866579,26855.695865


### Trend detection

In [3]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [4]:
def isPivot(candle, window):
    """
    函數用於檢測是否蠟燭是樞紐點/分形點
    參數：蠟燭索引，用於測試樞紐的蠟燭之前和之後的窗口大小
    返回值：如果是高點樞紐，則返回1，如果是低點樞紐，則返回2，如果兩者都是則返回3，默認情況下返回0
    """
    if candle - window < 0 or candle + window >= len(df):
        return 0

    pivotHigh = 1  # 假設蠟燭是高點樞紐
    pivotLow = 2  # 假設蠟燭是低點樞紐

    for i in range(candle - window, candle + window + 1):
        # 檢查蠟燭的低點是否比窗口內其他蠟燭的低點更低
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow = 0  # 如果有蠟燭的低點更低，則將低點樞紐設為0

        # 檢查蠟燭的高點是否比窗口內其他蠟燭的高點更高
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh = 0  # 如果有蠟燭的高點更高，則將高點樞紐設為0

    if (pivotHigh and pivotLow):
        return 3  # 如果既是高點樞紐又是低點樞紐，返回3
    elif pivotHigh:
        return pivotHigh  # 如果只是高點樞紐，返回1
    elif pivotLow:
        return pivotLow  # 如果只是低點樞紐，返回2
    else:
        return 0  # 如果都不是，返回0

In [5]:
window = 5  # 定義窗口大小

# 使用 lambda 函數將 isPivot 函數應用於 df 數據框的每一行
# 這裡的 x.name 表示蠟燭的索引，axis=1 表示按行應用函數
df['isPivot'] = df.apply(lambda x: isPivot(x.name, window), axis=1)

# 'isPivot' 列現在包含了每個蠟燭是否是樞紐點的結果

In [6]:
# 定義一個函數 pointpos，該函數根據 'isPivot' 列的值來計算點的位置
# 如果 'isPivot' 為2，表示低點樞紐，則返回該行 'low' 值減去一個小數值 1e-3
# 如果 'isPivot' 為1，表示高點樞紐，則返回該行 'high' 值加上一個小數值 1e-3
# 否則返回 NaN（缺失值）
def pointpos(x):
    if x['isPivot'] == 2:
        return x['low'] - 1e-3
    elif x['isPivot'] == 1:
        return x['high'] + 1e-3
    else:
        return np.nan

# 使用 apply 函數將 pointpos 函數應用於 df 數據框的每一行
# 結果將儲存在一個新的列 'pointpos' 中
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

# 'pointpos' 列現在包含了每個蠟燭對應樞紐點的位置，或者 NaN

In [7]:
# 從 df 數據框中選擇特定的行範圍，這裡是從索引300到索引449的範圍，包括這些行
dfpl = df[300:450]

# 創建一個新的圖形對象（Figure），其中包括蠟燭圖和樞紐點標記
fig = go.Figure(data=[
    go.Candlestick(x=dfpl.index,  # x 軸數據（時間）
                   open=dfpl['open'],  # 開盤價格
                   high=dfpl['high'],  # 最高價格
                   low=dfpl['low'],    # 最低價格
                   close=dfpl['close']  # 收盤價格
                  )
])

# 添加散點圖（Scatter plot）用於標記樞紐點的位置
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'],  # x 軸為時間，y 軸為樞紐點的位置
                mode="markers",  # 使用散點模式
                marker=dict(size=5, color="MediumPurple"),  # 散點的大小和顏色
                name="pivot"  # 散點圖的名稱
                )

# 更新圖形的布局，隱藏 x 軸上的區域選擇器（xaxis_rangeslider）
fig.update_layout(xaxis_rangeslider_visible=False)

# 顯示圖形
fig.show()

In [8]:
def detect_structure(candle, backcandles, window):
    """
    注意！窗口應始終大於樞紐窗口，以避免前瞻性偏見。
    """
    # 根據蠟燭索引計算需要的局部數據框範圍
    localdf = df[candle - backcandles - window:candle - window]

    # 獲取最近3個高點樞紐的高點價格和索引
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    idxhighs = localdf[localdf['isPivot'] == 1].high.tail(3).index

    # 獲取最近3個低點樞紐的低點價格和索引
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    idxlows = localdf[localdf['isPivot'] == 2].low.tail(3).index

    # 初始化一個變數，用於標記是否檢測到特定價格圖案
    pattern_detected = False

    # 定義一些閾值
    lim1 = 0.005
    lim2 = lim1 / 3

    if len(highs) == 3 and len(lows) == 3:
        # 檢查索引的順序是否符合條件
        order_condition = (idxlows[0] < idxhighs[0]
                           < idxlows[1] < idxhighs[1]
                           < idxlows[2] < idxhighs[2])

        # 檢查價格差異是否符合條件
        diff_condition = (
            abs(lows[0] - highs[0]) > lim1 and
            abs(highs[0] - lows[1]) > lim2 and
            abs(highs[1] - lows[1]) > lim1 and
            abs(highs[1] - lows[2]) > lim2
        )

        # 定義兩種可能的價格圖案
        pattern_1 = (lows[0] < highs[0] and
                     lows[1] > lows[0] and lows[1] < highs[0] and
                     highs[1] > highs[0] and
                     lows[2] > lows[1] and lows[2] < highs[1] and
                     highs[2] < highs[1] and highs[2] > lows[2])

        pattern_2 = (lows[0] < highs[0] and
                     lows[1] > lows[0] and lows[1] < highs[0] and
                     highs[1] > highs[0] and
                     lows[2] < lows[1] and
                     highs[2] < highs[1]
                     )

        # 如果所有條件都滿足，則標記價格圖案已被檢測到
        if (order_condition and
            diff_condition and
            (pattern_1 or pattern_2)
        ):
            pattern_detected = True

    # 如果檢測到價格圖案，則返回1，否則返回0
    if pattern_detected:
        return 1
    else:
        return 0

In [11]:
# 使用 map 函數將 detect_structure 函數應用於 df 數據框的每一個時間點（索引）
# detect_structure 函數檢測每個時間點是否存在特定的價格圖案
# backcandles 和 window 是 detect_structure 函數的參數
df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=6))

# 'pattern_detected' 列包含了每個時間點是否檢測到特定價格圖案的結果，1 表示檢測到，0 表示未檢測到

In [12]:
# 使用布林索引，篩選 'pattern_detected' 列的值不等於0的行
selected_rows = df[df['pattern_detected'] != 0]

# selected_rows 現在包含了 'pattern_detected' 列的值不等於0的時間點所對應的所有數據

# 你可以使用 selected_rows 來進一步分析或視覺化具有檢測到特定價格圖案的時間點的數據